Evet Bakalım Beauty Analysis Çalışalım biraz. 

In [8]:
import cv2
import matplotlib.pyplot as plt
import skimage.io as imageio
import numpy as np
import pandas as pd

In [107]:
mar=cv2.imread("./rl1.PNG")

In [ ]:
imageio.imshow(mar)

In [ ]:
rgb=cv2.cvtColor(mar,cv2.COLOR_BGR2RGB)
imageio.imshow(rgb)

In [13]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml

--2022-02-24 14:54:04--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 341406 (333K) [text/plain]
Saving to: ‘haarcascade_eye.xml’

haarcascade_eye.xml 100%[===================>] 333.40K  --.-KB/s    in 0.003s  

2022-02-24 14:54:04 (95.5 MB/s) - ‘haarcascade_eye.xml’ saved [341406/341406]



In [102]:
import dlib
#detector = dlib.get_frontal_face_detector()
eyedetect=cv2.CascadeClassifier('./haarcascade_eye.xml')


In [110]:
geteye=eyedetect.detectMultiScale(rgb)

In [111]:
geteye

array([[199,   4,  26,  26],
       [343, 214,  50,  50]], dtype=int32)

In [ ]:
gozler=[]
for (x,y,w,h) in geteye:
    gozler.append(rgb[y:y+h, x:x+w])
imageio.imshow(gozler[0])    

In [ ]:
imageio.imshow(gozler[1])

In [114]:
left_eye = gozler[0];
right_eye = gozler[1];

In [115]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import albumentations as A
from torch.utils.data import DataLoader,Dataset
from skimage import io
import pandas as pd
import cv2
import matplotlib.pyplot as plt


In [116]:
class CNN(nn.Module):

  def __init__(self,in_channels = 1,num_classes = 10):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size = (3,3), stride=(1,1), padding=(1,1)) # same convolution as output size will be same
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(4096,num_classes)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)

    return x

In [143]:
input_size = 64*64;
num_classes = 2;

model = CNN(input_size,num_classes)
model.load_state_dict(torch.load('model_95.pt',map_location=torch.device('cpu')));
model.eval()

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=2, bias=True)
)

In [144]:
transform = transforms.Compose([
                                 transforms.Resize(64),
                                 transforms.Grayscale(num_output_channels=1),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.5],std=[0.5],inplace=True)
                                ]);

In [ ]:
imageio.imshow(right_eye)

In [ ]:
from PIL import Image
import numpy as np

PIL_image = Image.fromarray(right_eye.astype('uint8'), 'RGB') #convery numpy to PIL image

t_img = transform(PIL_image).squeeze(); # squeeze used to change 1,64,64 to 64,64
imageio.imshow(t_img.numpy()) # convert back to numpy to display

In [147]:
img = transform(PIL_image);
img = img[None]
temp = model(img)


In [148]:
temp

tensor([[-2.0020,  1.1284]], grad_fn=<AddmmBackward0>)

In [ ]:

import cv2;
import numpy as np;
import dlib
from google.colab.patches import cv2_imshow
#detector = dlib.get_frontal_face_detector()
eyedetect=cv2.CascadeClassifier('./haarcascade_eye.xml')



# font
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (50, 50)
# fontScale
fontScale = 1
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2

  
# define a video capture object
vid = cv2.VideoCapture('')
  
while(True):
      
    # Capture the video frame
    # by frame
    #ret, frame = vid.read()
    frame=cv2.imread("./rl1.PNG")

    if(frame.all() != None):

      geteye=eyedetect.detectMultiScale(frame)
      
      eyes=[]
      for (x,y,w,h) in geteye:
          eyes.append(rgb[y:y+h, x:x+w])
      
      imageio.imshow(eyes[1])    
      img = eyes[1]
      img = Image.fromarray(img.astype('uint8'), 'RGB')
      img = transform(img);
      img = img[None]
      scores = model(img)

      print(scores)
      prediction = scores[0].argmax()

      frame = cv2.putText(frame, str(prediction), org, font, fontScale, color, thickness, cv2.LINE_AA)    
    
    #cv2.imshow('frame', frame)
    cv2_imshow(frame);
      
    break;
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

In [ ]:

import cv2;
import numpy as np;
import dlib
from google.colab.patches import cv2_imshow
#detector = dlib.get_frontal_face_detector()
eyedetect=cv2.CascadeClassifier('./haarcascade_eye.xml')



# font
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (50, 50)
# fontScale
fontScale = 1
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2

  
# define a video capture object
vid = cv2.VideoCapture('')
  
while(True):
      
    # Capture the video frame
    # by frame
    #ret, frame = vid.read()
    frame=cv2.imread("./rl1.PNG")

    if(frame.all() != None):

      geteye=eyedetect.detectMultiScale(frame)
      
      eyes=[]
      for (x,y,w,h) in geteye:
          eyes.append(rgb[y:y+h, x:x+w])
      
      imageio.imshow(eyes[0])    
      img = eyes[0]
      img = Image.fromarray(img.astype('uint8'), 'RGB')
      img = transform(img);
      img = img[None]
      scores = model(img)

      print(scores)
      prediction = scores[0].argmax()

      frame = cv2.putText(frame, str(prediction), org, font, fontScale, color, thickness, cv2.LINE_AA)    
    
    #cv2.imshow('frame', frame)
    cv2_imshow(frame);
      
    break;
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()